In [1]:
import sys, os


src = r"C:\Users\user\Desktop\Coding mo\AutoML proj1\task 1\AutoML Project Molham\src" # copy the location of the folder that has paths.py file using (ctrl + shift + C ) and paste it here    
sys.path.append(src)

from paths import *

get_paths()

sys.path.append(SRC_PATH)

from pipeline import *




In [2]:
import pickle

with open(os.path.join(ABLATION_MODELS_FOR_SHAP_PATH, "all_results.pkl"), "rb") as f:
    all_results = pickle.load(f)

with open(os.path.join(ABLATION_MODELS_FOR_SHAP_PATH, "best_models_dict.pkl"), "rb") as f:
    best_models_dict = pickle.load(f)

with open(os.path.join(ABLATION_MODELS_FOR_SHAP_PATH, "X_dict.pkl"), "rb") as f:
    X_dict = pickle.load(f)

with open(os.path.join(ABLATION_MODELS_FOR_SHAP_PATH, "feature_names_dict.pkl"), "rb") as f:
    feature_names_dict = pickle.load(f)


In [3]:
import shap

def run_simple_explainability(all_results, best_models_dict, X_dict, feature_names_dict, max_rows=500, 
                              top_features_count=3, force_samples_count=2):

    df_results = pd.DataFrame(all_results)

    for dataset_name in df_results['dataset'].unique():
        for fold in df_results[df_results['dataset']==dataset_name]['fold'].unique():


            def predict_fn(X_np):
                X_df = pd.DataFrame(X_np, columns=X_sample.columns)
                return model.predict(X_df).data[:, 0]


            
            fold_df = df_results[(df_results['dataset']==dataset_name) & (df_results['fold']==fold)]
            best_row = fold_df.loc[fold_df['f1'].idxmax()]
            best_ablation = best_row['ablation']

            


            
            model_path = best_models_dict[dataset_name][f"fold_{fold}"][best_ablation]
            with open(model_path, "rb") as f:
                model = pickle.load(f)


            
            X_val = X_dict[dataset_name][f"fold_{fold}"].copy()
            feature_names_after = feature_names_dict[dataset_name][f"fold_{fold}"][best_ablation]["after"]
            if X_val.shape[1] != len(feature_names_after):
                feature_names_after = [f"f_{i}" for i in range(X_val.shape[1])]
            X_val.columns = feature_names_after

            
            X_sample = X_val.sample(n=min(max_rows, X_val.shape[0]), random_state=42)
            X_sample_np = X_sample.to_numpy()

            
            model_name = f"{dataset_name}_fold{fold}_{best_ablation}"


            



            
            try:
                background = X_sample_np[:min(100, X_sample_np.shape[0]), :]
                explainer = shap.Explainer(predict_fn, background)
                shap_values = explainer(X_sample_np)
                
            except Exception as e:
                continue




            
            try:
                summary_path = os.path.join(FEATURE_IMPORTANCE_SHAP_PATH, f"{model_name}_summary.png")
                shap.summary_plot(shap_values.values, X_sample, feature_names=X_sample.columns, show=False)
                plt.savefig(summary_path, bbox_inches="tight")
                plt.close()
                
            except:
                continue




            
            try:
                bar_path = os.path.join(FEATURE_IMPORTANCE_SHAP_PATH, f"{model_name}_importance.png")
                shap.summary_plot(shap_values.values, X_sample, feature_names=X_sample.columns, plot_type="bar", show=False)
                plt.savefig(bar_path, bbox_inches="tight")
                plt.close()

            
            except Exception as e:
                continue

            top_features = X_sample.columns[:min(top_features_count, X_sample.shape[1])]


            
            for f in top_features:
                try:
                    dep_path = os.path.join(PER_FEATURE_SHAP_PATH, f"{model_name}_dependence_{f}.png")
                    shap.dependence_plot(f, shap_values.values, X_sample, show=False)
                    plt.savefig(dep_path, bbox_inches="tight")
                    plt.close()
                except Exception as e:
                    continue



            

            
            for i in range(min(force_samples_count, X_sample.shape[0])):

                try:
                    sample = X_sample.iloc[i, :]
                    sample_explainer = shap.Explainer(predict_fn, X_sample_np[:min(100, X_sample_np.shape[0]), :])
                    sample_shap = sample_explainer(sample.to_frame().T)

                    force_path = os.path.join(SAMPLES_SHAP_PATH, f"{model_name}_force_{i}.png")
                    shap.plots.force(sample_shap, matplotlib=True, show=False)
                    plt.savefig(force_path, bbox_inches="tight")
                    plt.close()

                
                except Exception as e:
                    continue







run_simple_explainability(all_results=all_results, best_models_dict=best_models_dict, X_dict=X_dict, feature_names_dict=feature_names_dict, max_rows=20000)








'nlp' extra dependency package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


PermutationExplainer explainer: 20001it [1:25:22,  3.90it/s]                           
PermutationExplainer explainer: 20001it [1:26:20,  3.85it/s]                           
PermutationExplainer explainer: 90it [01:38,  1.23s/it]                        
PermutationExplainer explainer: 90it [01:45,  1.30s/it]                        
